In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("./data/train.csv")

In [ ]:
data.info()

# 노드 특성(Node features)
* 도서 정보: 평점, 제목, 저자, 출판년도, 출판사, 장르(**←추후 NLP 기술을 통해 책을 장르별로 분류해 feature 생성 가능**) 등
* 사용자 정보: 나이, 지역, 장르별 관심도(**←추후 NLP 기술을 통해 유저별 장르 관심도를 수치화해 feature로 생성 가능**) 등

In [ ]:
from gensim.models import KeyedVectors

# 사전 훈련된 Word2Vec 모델 로드
word2vec = KeyedVectors.load_word2vec_format('path/to/word2vec_model.bin', binary=True)

# 문자열 데이터를 벡터로 변환하는 함수
def string_to_vector(text):
    words = text.split()
    word_vectors = [word2vec[word] for word in words if word in word2vec.vocab]
    if len(word_vectors) == 0:
        return np.zeros(word2vec.vector_size)
    return np.mean(word_vectors, axis=0)

# 제목, 저자, 출판사 데이터를 벡터로 변환
data['Title-Vec'] = data['Book-Title'].apply(string_to_vector)
data['Author-Vec'] = data['Book-Author'].apply(string_to_vector)
data['Publisher-Vec'] = data['Publisher'].apply(string_to_vector)

# 벡터로 변환된 데이터를 사용하여 노드 피처를 구성
node_features = data[['User-ID', 'Book-ID', 'Title-Vec', 'Author-Vec', 'Publisher-Vec']]

# 엣지 특성(Edge features):
* 사용자 간 관계: 같은 지역
* 사용자와 도서 간 관계: 평점

In [ ]:
# 그래프 생성
G = nx.Graph()

# 노드 및 엣지 추가 (예: data 변수를 사용한다고 가정)
# ...

# 지역 정보를 기반으로 사용자 간 엣지 추가
user_location = data[['User-ID', 'Location']].drop_duplicates()
location_groups = user_location.groupby('Location')['User-ID'].apply(list)

for _, user_list in location_groups.iteritems():
    if len(user_list) > 1:
        for i in range(len(user_list) - 1):
            for j in range(i + 1, len(user_list)):
                G.add_edge(user_list[i], user_list[j], weight=1)


# 기초 통계량(Basic statistics)
* 도서 수, 사용자 수
* 사용자별 도서 평가 수, 나이 분포, 평균 평점 등
* 장르별 도서 수, 평균 평점, 리뷰 수 등 (**←추후 feature 생성하고 진행**)

In [ ]:
# 기초 통계량 계산
unique_users = data['User-ID'].nunique()
unique_books = data['Book-ID'].nunique()
ratings_per_user = data.groupby('User-ID')['Book-Rating'].count()
age_distribution = data['Age'].value_counts()
average_rating = data['Book-Rating'].mean()

# 기초 통계량 출력
print(f"사용자 수: {unique_users}")
print(f"도서 수: {unique_books}")
print(f"사용자별 도서 평가 수:")
print(ratings_per_user.describe())
print(f"나이 분포:")
print(age_distribution.head(10))  # 상위 10개 출력
print(f"평균 평점: {average_rating}")

# 결측치 및 이상치(Missing & Outliers)
* 결측치: 도서 정보나 사용자 정보의 누락된 값 확인    
    ㄴ 없음
* 이상치: 예상치와 크게 다른 값들, 도서 평점의 분포 등

In [ ]:
# 도서 평점 분포 확인
plt.figure(figsize=(10, 5))
sns.countplot(data['Book-Rating'])
plt.title('도서 평점 분포')
plt.xlabel('평점')
plt.ylabel('빈도')
plt.show()

# 이상치 탐지: IQR 방식
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# 이상치 탐지: 평균과 표준편차 방식
def detect_outliers_std(data, column, threshold=3):
    mean = data[column].mean()
    std = data[column].std()
    return data[np.abs(data[column] - mean) > (threshold * std)]

# 이상치 탐지 결과 출력 (예: Age 열)
outliers_iqr = detect_outliers_iqr(data, 'Age')
outliers_std = detect_outliers_std(data, 'Age')

print(f"IQR 방식으로 검출된 이상치 개수: {len(outliers_iqr)}")
print(f"평균과 표준편차 방식으로 검출된 이상치 개수: {len(outliers_std)}")

# 그래프 구조(Graph structure)
* 연결 밀도(Density): 전체 가능한 연결 중 실제 연결 비율    
* 중심성(Centrality): 중요한 노드를 파악하는 지표 (차수 중심성, 근접 중심성, 매개 중심성 등)
* 커뮤니티(Community): 서로 밀접하게 연결된 노드 그룹
* 직경(Diameter) 및 평균 최단 경로 길이(Average shortest path length): 그래프의 전체 연결 정도를 나타냄

In [ ]:
G = nx.Graph()

# 노드 추가
G.add_nodes_from(data['User-ID'].unique(), node_type='user')
G.add_nodes_from(data['Book-ID'].unique(), node_type='book')

# 엣지 추가
edges = [(row['User-ID'], row['Book-ID'], {'rating': row['Book-Rating']}) for _, row in data.iterrows()]
G.add_edges_from(edges)

# 그래프 정보 출력
print(f"노드 개수: {G.number_of_nodes()}")
print(f"엣지 개수: {G.number_of_edges()}")

# 연결 밀도(Density)
density = nx.density(G)
print(f"연결 밀도(Density): {density}")

# 중심성(Centrality)
degree_centrality = nx.degree_centrality(G)

# 그래프 시각화 (선택 사항)
# nx.draw(G, with_labels=True, node_size=30, font_size=8)
# plt.show()

In [ ]:
# 특성 간 상관관계(Correlations between features)
* 도서 특성 간 상관관계: 장르별 평균 평점, 장르별 판매량 등
* 사용자 특성 간 상관관계: 나이와 관심 장르, 성별과 평점 분포 등